In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Some imports**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
import optuna
from sklearn import metrics 
from imblearn.over_sampling import SMOTE
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold, cross_validate, learning_curve
from functools import partial
from skopt import space
from sklearn import ensemble
from skopt import gp_minimize
from collections import Counter
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt 
import plotly.express as px
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df= pd.read_csv("/kaggle/input/credit-card-customers/BankChurners.csv")

# 4. Data Preprocessing


In [ ]:
df.head(5)

In [ ]:
print(df.isnull().sum().sort_values(ascending=False))

In [ ]:
df.describe()

#### Categorical Data Handling

In [ ]:
df['Gender'] = [1 if x == 'M' else 0 for x in df['Gender']]
df['Attrition_Flag'] = [1 if x == 'Existing Customer' else 0 for x in df['Attrition_Flag']]
df

In [ ]:
df['Income_Category'].value_counts()

In [ ]:
df['Income_Category'] = df['Income_Category'].replace({'Unknown': 0 , 'Less than $40K':1, '$40K - $60K':2,'$80K - $120K':3, '$60K - $80K':4, '$120K +':5})
df['Education_Level'] = df['Education_Level'].replace({'Unknown': 0, 'High School':1, 'Graduate':2, 'Uneducated':3,'College':4,'Post-Graduate':5,'Doctorate':6})
df

In [ ]:
df=pd.get_dummies(df)
df

In [ ]:
df=df.drop(['CLIENTNUM','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], 1)

In [ ]:
plt.figure(figsize = (20,20))
sns_plot=sns.heatmap(df.corr(),annot= True)
plt.show()

In [ ]:
y = df['Attrition_Flag']
X = df.drop(['Attrition_Flag'], axis =1)

In [ ]:
X_train,X_test,y_train,y_test =train_test_split(X,y , test_size=0.25, random_state=42)

# PCA

In [ ]:
scaler=MinMaxScaler()
scaler.fit(X_train)
scaled_train=scaler.transform(X_train)
scaled_test=scaler.transform(X_test)

In [ ]:
pca = PCA()
pct = pca.fit_transform(scaled_train)
print(len(pca.explained_variance_ratio_))
a=[print('the sum of %d components: ' %i , sum(pca.explained_variance_ratio_[:i])) for i in range(0,len(pca.explained_variance_ratio_),3)]

In [ ]:
pca = PCA(n_components=15)
pct = pca.fit_transform(scaled_train)

In [ ]:
principal_df = pd.DataFrame(pct,columns=['pc'+str(i) for i in range(len(pca.explained_variance_))])
principal_df=principal_df.reset_index(drop=True)

Label_df=pd.DataFrame(y_train)
Label_df=Label_df.reset_index(drop=True)

finaldf= pd.concat([principal_df,Label_df],axis=1)
finaldf

In [ ]:
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}
labels

In [ ]:
fig = px.scatter_matrix(
    pct,
    labels=labels,
    dimensions=range(4),
    color=finaldf["Attrition_Flag"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

# SMOTE

## data discribtion before Balancing

In [ ]:
counter = Counter(y)
counter

In [ ]:
 for k,v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

## Repeat data discribtion after SMOTE

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [ ]:
counter = Counter(y)
for k,v in counter.items():
	per = v / len(y) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show

# Modelling

In [ ]:
X_train,X_test,y_train,y_test =train_test_split(X,y , test_size=0.20, random_state=42)

In [ ]:
X_train = X_train.values
y_train = y_train.values

In [ ]:
X_test = X_test.values
y_test = y_test.values

### Random Forest Classifier

In [ ]:
model = RandomForestClassifier(criterion= 'gini', max_depth= 15, max_features= 0.2, n_estimators= 600, n_jobs= -1, random_state=0)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test,y_pred)

In [ ]:
disp = plot_confusion_matrix(model, X_test, y_test,
                              display_labels=['Existing ','Attrited '],
                              cmap=plt.cm.Blues)

### GradientBoost

In [ ]:
gboost = ensemble.GradientBoostingClassifier(n_estimators=961,max_depth=5,learning_rate=0.4157338892518516,max_features='auto',random_state=19,verbose=0)

In [ ]:
gboost.fit(X_train,y_train)

In [ ]:
gboost_pred = gboost.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test,gboost_pred)

In [ ]:
disp = plot_confusion_matrix(gboost, X_test, y_test,
                              display_labels=['Existing','Attrited'],
                              cmap=plt.cm.Blues)

### Adaboost

In [ ]:
Ada_model = ensemble.AdaBoostClassifier(n_estimators=926,learning_rate=0.6513888777930883,random_state=17)

In [ ]:
Ada_model.fit(X_train,y_train)

In [ ]:
Ada_pred = Ada_model.predict(X_test)

In [ ]:
metrics.accuracy_score(y_test,Ada_pred)

In [ ]:
disp = plot_confusion_matrix(Ada_model, X_test, y_test,
                              display_labels=['Existing','Attrited'],
                              cmap=plt.cm.Blues)